# CTR Mode (Counter Mode)

CTR mode converts a block cipher (e.g., AES) into a stream cipher. Instead of directly encrypting message blocks, it generates a keystream by encrypting sequential counter values combined with a nonce, then XORs this keystream with the plaintext. Since encryption and decryption are symmetric (both use XOR), the same algorithm applies for both processes.

---

## Key Characteristics

- **Parallelizable:**  
  Multiple blocks can be processed in parallel, making CTR mode faster than modes like CBC.
  
- **No Padding Needed:**  
  As a stream cipher, plaintext does not require padding.
  
- **Identical Operations:**  
  Encryption and decryption are performed by the same XOR operation using the keystream.

---

## How It Works

1. **Keystream Generation:**  
   - A unique **nonce** (or IV) is generated for each encryption.
   - A **counter** is used (commonly starting at 0) and incremented for each block.
   - For each block, the cipher computes:  
     ```
     Keystream_i = E_K(Nonce || Counter_i)
     ```
     where `E_K` is the block cipher encryption with key K.

2. **Encrypting/Decrypting Data:**  
   - **Encryption:**  
     ```
     Ciphertext = Plaintext ⊕ Keystream
     ```
   - **Decryption:**  
     ```
     Plaintext = Ciphertext ⊕ Keystream
     ```
   Since XOR is its own inverse, the same operation recovers the plaintext.

---

## Communication Flow

```text
Alice                               Bob                    Eve
-----                               ---                    ---
Plaintext P
   | -- Encrypt using CTR with key K and nonce N --> Ciphertext C
   |                                                   (Eve sees C and N)
   | -- Decrypt using same key K and nonce N --> Plaintext P
```

- **Eve’s Limitations:**  
  Without knowing the secret key K, having access to the nonce N and ciphertext C does not reveal any information about the plaintext.

---

## Parameters

- **Key (K):**  
  Can be AES-128, AES-192, or AES-256.

- **Nonce (IV):**  
  A random, unique value for each encryption; it must never be reused with the same key.  
  *Tip:* The nonce and counter are typically concatenated to form the block input.

- **Counter:**  
  A value that is incremented for each block; ensures that each keystream block is unique.

- **Block Size:**  
  128 bits for AES.

---
## Encryption
![CTR Encryption](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/CTR_encryption_2.svg/601px-CTR_encryption_2.svg.png)

---
## Decryption

![CTR Decryption](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/CTR_decryption_2.svg/601px-CTR_decryption_2.svg.png)

---

## Security Considerations

- **Advantages:**  
  - High performance due to parallelization.  
  - Eliminates padding overhead.

- **Critical Requirements:**  
  - **Unique Nonce:** Reusing a nonce with the same key leads to keystream reuse, which can catastrophically compromise security.
  - **No Built-In Integrity:**  
    CTR mode does not provide message authentication. Always combine it with a MAC (or use an AEAD mode like AES-GCM) for integrity.
  - ⚠️ If nonce is reused with the same key, attacker can XOR two ciphertexts to cancel keystream → dangerous!
---

## Real-World Applications

- **TLS/HTTPS:**  
  Often used as part of the encryption process (e.g., AES-CTR or as a basis for AES-GCM).
  
- **Disk Encryption:**  
  Suitable for streaming encryption of files or disks.
  
- **Foundation for AEAD Modes:**  
  CTR mode underpins advanced modes like GCM, which combine encryption and authentication.

---

**Summary:**  
CTR mode is a versatile and efficient encryption mode that turns block ciphers into stream ciphers. Its speed and parallelizability make it ideal for high-throughput applications, provided that nonce reuse is strictly avoided and additional measures (like MACs) are implemented for integrity.

In [1]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

# --- Key (AES-128 = 16 bytes) ---
key = get_random_bytes(16)

# --- Nonce (must be unique for each encryption with same key) ---
nonce = get_random_bytes(8)  # CTR uses nonce+counter internally

# --- Cipher (encrypt) ---
cipher_enc = AES.new(key, AES.MODE_CTR, nonce=nonce)

plaintext = b"CTR mode demo: works like a stream cipher"
ciphertext = cipher_enc.encrypt(plaintext)
print("Ciphertext (hex):", ciphertext.hex())

# --- Cipher (decrypt) ---
cipher_dec = AES.new(key, AES.MODE_CTR, nonce=nonce)
recovered = cipher_dec.decrypt(ciphertext)
print("Recovered:", recovered)


Ciphertext (hex): 63737293f0bc6ba47835390eba9c65ac9a5bfd3c021eac397b6719803b67ef35494580845eb9425e58
Recovered: b'CTR mode demo: works like a stream cipher'
